In [ ]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# math
import numpy as np
import torch
import pytorch3d
import pytorch3d.transforms
# physics
import warp as wp
import warp.sim as wps
# plots
import matplotlib as mpl
from matplotlib import pyplot as plt
from src.mpl_utils import set_fig_size, set_mpl_format
# config
from src.config import cfg
# dataset
from src.dataset import load_train_data
from src.data.visualize import print_example

# setup
set_mpl_format()
wp.init()

### Config

In [ ]:
# fill in your directory set up here
config_fp = os.path.join(project_dir, f'cfg/templates/comp-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.DATA.path = os.path.join(project_dir, 'data/sets/')
print(cfg)

### Data

In [ ]:
train_ds, ds_info = load_train_data(cfg)
example = next(iter(train_ds))

In [ ]:
vid_id = example['metadata']['video_name']
obj_count = example['metadata']['num_instances']
print(f'Video: {vid_id}, Num objects: {obj_count}')

In [ ]:
example['instances']['velocities'].shape

In [ ]:
example['instances'].keys()

### Scene

In [ ]:
# simulation parameters
sim_duration = 1.0
# control frequency
fps = 12
frame_dt = 1.0 / float(fps)
frame_steps = int(sim_duration / frame_dt)
# sim frequency
sim_substeps = 8 * 5
sim_steps = frame_steps * sim_substeps
sim_dt = frame_dt / sim_substeps

### Physics

In [ ]:
CLEVR_SHAPE_NAMES = ("cube", "cylinder", "sphere")
CLEVR_SIZE_NAMES = ('small', 'large')
CLEVR_SIZES = (0.7, 1.4)
CLEVR_DENSITIES = (1.1, 2.7)

phys_device = wp.get_cuda_device()
phys_builder = wps.ModelBuilder(up_vector=(0, 0, 1))

ground_mu = 0.3
ground_restitution = 0.5

# ground
phys_builder.set_ground_plane(mu=ground_mu, 
                              restitution=ground_restitution)

for obj_idx in range(obj_count):
    obj_class_idx = example['instances']['shape_label'][obj_idx]
    obj_class = CLEVR_SHAPE_NAMES[obj_class_idx]
    
    obj_scale_idx = example['instances']['size_label'][obj_idx]
    obj_scale = CLEVR_SIZES[obj_scale_idx] / 2.0
    obj_pos = example['instances']['positions'][obj_idx][0]
    obj_ori = example['instances']['quaternions'][obj_idx][0]

    obj_vel = example['instances']['velocities'][obj_idx][0]

    obj_material = example['instances']['material_label'][obj_idx]
    obj_density = CLEVR_DENSITIES[obj_material]

    obj_mass = float(example['instances']['mass'][obj_idx])
    obj_fric = example['instances']['friction'][obj_idx]
    obj_restitution = example['instances']['restitution'][obj_idx]

    phys_builder.add_particle(
        pos=obj_pos,
        vel=obj_vel,
        radius=obj_scale,
        mass=obj_mass,
        
    )

phys_model = phys_builder.finalize(phys_device, requires_grad=False)

phys_states = [phys_model.state(requires_grad=False) for _ in range(sim_steps + 1)]

In [ ]:
def capture_phys(device, model, integrator, states):
    # tape tracks computation graph
    tape = wp.Tape()
    wp.capture_begin(device=device)
    with tape:
        for i in range(sim_steps):
            states[i].clear_forces()
            wp.sim.collide(model, states[i])
            integrator.simulate(model, states[i], states[i + 1], sim_dt)
    graph = wp.capture_end()

    return graph, tape


def get_trajectory(states):
    trajectory = []
    for state in states[::sim_substeps]:
        trajectory.append(state.particle_q.numpy().squeeze())
    return np.array(trajectory)


def distance(target_pose, pred_pose):
    return torch.linalg.norm(target_pose - pred_pose)

In [ ]:
# initial run 
phys_integrator = wp.sim.SemiImplicitIntegrator()
phys_graph, phys_tape = capture_phys(phys_device, phys_model, phys_integrator, phys_states)
wp.capture_launch(phys_graph)

In [ ]:
positions = np.zeros_like(example['instances']['positions'])
quaternions = np.zeros_like(example['instances']['quaternions'])

for i in range(len(phys_states)):
    pos_ori = phys_states[i].body_q.numpy()
    positions[i] = pos_ori[i, :3]
    quaternions[i] = pos_ori[i, 3:]